In [2]:
import torch
from gensim.models import KeyedVectors
import pandas as pd

from dataset import MyDataset
from han import Han

In [3]:
wv = KeyedVectors.load("embedding/yelp.wv")
model = Han(wv.vectors, num_classes=5)

In [26]:
for k, v in zip(model.state_dict().items(), model.parameters()):


<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'>
<class 'torch.Tensor'>
<class 'torch.nn.parameter.Parameter'>
<class 'str'